# Scheduling Script

This script uses the python package "mip" to schedule the order of irrigation.

The constraints for this schedule are given through two types of settings: common and custom. Common settings are the "default" settings given to the fields of a farm at the outset and are the reversion settings if a reset is required. Custom settings are field-level customizations to one or more of the common settings. Below are the fields in the "CurrentConstraints" table in Azure.

- field_id
- setting_id
- max_sprinkle_dur - the maximum duration of irrigation for the field
- offset_bef_sprinkle - 
- max_wind_speed - the maximum allows wind speed for irrigation 
- humid - the maximum humidity allowed for irrigation
- skip_sprinkle_from - the starting time of an irrigation blackout 
- skip_sprinkle_to - the ending time of an irrigation blackout
- last_updated

# Import packages and data

In [1]:
from mip import *
import numpy as np
import pandas as pd
from azure.cosmosdb.table.tableservice import TableService
import creds
import math
import datetime

table_service = TableService(account_name=creds.ACCNAME, account_key=creds.KEY)

In [2]:
def get_df(table):
#     now = datetime.datetime.now() - datetime.timedelta(days = 2, hours = 1)
#     datefilter = "RowKey ge '" + str(now.timestamp()) + "'"
#     weather_gen = table_service.query_entities(table, filter=datefilter)
    gen = table_service.query_entities(table)
    to_df = []
    for row in gen:
        to_df.append(row)
    df = pd.DataFrame(to_df)
    return df

In [3]:
constraints = get_df('CurrentConstraints').sort_values('RowKey')
constraints = constraints.reset_index(drop=True).drop(['PartitionKey','RowKey','Timestamp','etag','last_updated'], axis=1)

predictions = get_df('Predictions').sort_values('RowKey')
predictions = predictions.reset_index(drop=True).drop(['PartitionKey','RowKey','Timestamp','etag'], axis=1)
predictions = predictions.iloc[-2:]

In [17]:
predictions[['D0_avgtemp_c','D0_mintemp_c','D0_maxtemp_c']]

,D0_avgtemp_c,D0_mintemp_c,D0_maxtemp_c
0,28.0,26.0,31.0
1,28.0,26.0,31.0


In [4]:
constraints

,field_id,humid,max_sprinkle_dur,max_wind_speed,offset_bef_sprinkle,setting_id,skip_sprinkle_from,skip_sprinkle_to
0,101,60.0,50,23,9,1,2,20
1,102,60.0,50,23,9,1,2,20


# Build data

In [ ]:
fields = constraints.shape[0]
field_idx = [i for i in range(fields)]
pred_name = 'pred6hrain'
start_time = 9
end_time = 18
slot_dur = 5
min_sm = 50
time_horizon = (end_time-start_time)*60

slots = int(time_horizon / slot_dur)
slots_hour = int(slots / start_time)
slot_idx = range(slots) # slot 0 begins at the start time

# For inputs
# Need another variable for starting soil moisture
begin_sm = [float(predictions.iloc[i]['humidity']) for i in field_idx] # needs to be changed to the actual soil moisture reading
# predicted_moisture[field_id] # Predictions based on no water being irrigated to it but assuming rain
pred_sm = [int(float(predictions.iloc[i][pred_name])) for i in field_idx]
# Do we need to triangulate this forecast?
wind = [float(predictions.iloc[i]['D0_maxwind_kph']) for i in field_idx]
# Do we need to triangulate this forecast?
humidity = [float(predictions.iloc[i]['D0_avghumidity']) for i in field_idx]

# For constraints
max_sprinkle_dur = [int(constraints.iloc[i]['max_sprinkle_dur']) for i in field_idx]
offset_bef_sprinkle = [int(constraints.iloc[i]['offset_bef_sprinkle']) for i in field_idx]
max_wind_speed = [int(constraints.iloc[i]['max_wind_speed']) for i in field_idx]
max_humid = [int(constraints.iloc[i]['humid']) for i in field_idx]
skip_sprinkle_from = [int(constraints.iloc[i]['skip_sprinkle_from']) for i in field_idx]
skip_sprinkle_to = [int(constraints.iloc[i]['skip_sprinkle_to']) for i in field_idx]

humid_flag = [1 if max_humid[i] > humidity[i] else 0 for i in field_idx] # do not irrigate if flag = 0
# temp_flag = [1 if (max_temp[i]-temp[i]) > 0 else 0 for i in field_idx] # do not irrigate if flag = 0
wind_flag = [1 if max_wind_speed[i] > wind[i] else 0 for i in field_idx] # do not irrigate if flag = 0

# Need another variable for soil moisture gained per slot; same process as "loss" but for days with irrigation
sm_gain = (9 / (24*12)) * 100
# Need another variable for soil moisture lost per slot # don't need; can just take from the beginning and ending moisture to derive # Take sum of all deltas between current day and tomorrow's soil moisture and divide by 24 hours * time slots in hour
sm_loss = (1.8 / (24*12)) * 100 # Not sure we can use this, since we also need to know both when rain will begin and the gestation period for sensors to detect water

slots_req = [min(max_sprinkle_dur[i]//slot_dur,math.ceil((min_sm - pred_sm[i])/sm_gain)) for i in field_idx] # Takes the max sprinkle duration or the total slots required to bring the soil moisture to an acceptable level

## Testing data

In [4]:
from mip import *
import random
import math
import datetime

fields = 10
field_idx = [i for i in range(fields)]
# pred_name = 'pred6hrain'
start_time = 9
end_time = 18
slot_dur = 5
min_sm = 50
time_horizon = (end_time-start_time)*60

slots = int(time_horizon / slot_dur)
slots_hour = int(slots / start_time)
slot_idx = range(slots) # slot 0 begins at the start time

# For inputs
# Need another variable for starting soil moisture
begin_sm = random.sample(range(50,61), fields)
# predicted_moisture[field_id] # Predictions based on no water being irrigated to it but assuming rain
pred_sm = random.sample(range(30,50), fields)
# Do we need to triangulate this forecast?
wind = [0 for _ in field_idx]
# Do we need to triangulate this forecast?
humidity = [50 for _ in field_idx]

# For constraints
max_sprinkle_dur = random.sample([10,10,10,10,10,10,10,10,10,10,15,15,15,15,15,15,15,15,15,15,20,20,20,20,20,20,20,20,20,20], fields)
offset_bef_sprinkle = [5 for _ in field_idx] # The time it takes for water to arrive at the parcel once pump activated
max_wind_speed = [50 for _ in field_idx]
max_humid = [75 for _ in field_idx]
skip_sprinkle_from = [11 for _ in field_idx]
skip_sprinkle_to = [14 for _ in field_idx]

temperature_range = [32, 35]
temp_soft_flag = 1 if (float(predictions['D0_mintemp_c'][predictions.shape[0]-1]) >= temperature_range[0]) & (float(predictions['D0_maxtemp_c'][predictions.shape[0]-1]) <= temperature_range[1]) else 0

cloudy_flag = 1 # If cloudy that day
humid_flag = [1 if max_humid[i] > humidity[i] else 0 for i in field_idx] # do not irrigate if flag = 0
# temp_flag = [1 if (max_temp[i]-temp[i]) > 0 else 0 for i in field_idx] # do not irrigate if flag = 0
wind_flag = [1 if max_wind_speed[i] > wind[i] else 0 for i in field_idx] # do not irrigate if flag = 0
temp_flag = 0 if (float(predictions['D0_mintemp_c'][predictions.shape[0]-1]) >= temperature_range[1]) | (float(predictions['D0_maxtemp_c'][predictions.shape[0]-1]) <= temperature_range[0]) else 1


# Need another variable for soil moisture gained per slot; same process as "loss" but for days with irrigation
sm_gain = (9 / (24*12)) * 100 # 9 is random
# Need another variable for soil moisture lost per slot # don't need; can just take from the beginning and ending moisture to derive # Take sum of all deltas between current day and tomorrow's soil moisture and divide by 24 hours * time slots in hour
sm_loss = (1.8 / (24*12)) * 100 # Not sure we can use this, since we also need to know both when rain will begin and the gestation period for sensors to detect water

slots_req = [min(max_sprinkle_dur[i]//slot_dur,math.ceil((min_sm - pred_sm[i])/sm_gain)) for i in field_idx]

# Create model

In [5]:
m = Model()
m = Model(sense=MINIMIZE)

In [6]:
# Decision variables in readable form
x = [[m.add_var(name='field {} irrigating slot {}:{}'.format(f+1, start_time+s//slots_hour,str((s%slots_hour)*slot_dur+100)[1:]), var_type=BINARY) for s in slot_idx] for f in field_idx] # 1 if field f is irrigated at slot s and 0 otherwise
y = [[m.add_var(name='field {} start slot {}:{}'.format(f+1, start_time+s//slots_hour,str((s%slots_hour)*slot_dur+100)[1:]), var_type=BINARY) for s in slot_idx] for f in field_idx] # 1 if field f starts irrigating at slot s and 0 otherwise

## Constraints

A few constraints still need to be added:
- Irrigate when the temperature is between 26-29; this may require having more granular temperature levels and then slowly ramping up between "checkpoints" (i.e. smooth the increase between two hourly temperature predictions by dividing with the slot_dur)
- Include offset to the irrigation system. However, it currently isn't clear how this should/will impact the model. Should it be added to the overall duration? Should it shift the slots down by one? etc

In [7]:
# Subject to
# No irrigation between skip times unless cloudy
for f in field_idx:
    m += xsum(x[f][s] for s in range((skip_sprinkle_from[f]-start_time)*slots_hour,(skip_sprinkle_to[f]-start_time)*slots_hour)) <= temp_soft_flag*cloudy_flag*slots

# Only water if prediction will fall below soil moisture threshold
for f in field_idx:
    m += xsum(x[f][s] for s in slot_idx) >= slots_req[f] * humid_flag[f]

# Irrigation should be done contiguously and not spread across a time horizon
for f in field_idx:
    m += xsum(y[f][s] for s in slot_idx) <= 1
    m += xsum(x[f][s] for s in slot_idx) <= xsum(y[f][s] for s in slot_idx)*slots
    for s in slot_idx:
        m += x[f][s] >= y[f][s]
# The following constraint needs to be optimized for allowing a variable end 
for f in field_idx:
    for s in slot_idx:
        m += slots_req[f]*y[f][s] <= xsum(x[f][u] for u in range(s,min(slots,s+slots_req[f])))

# At most one field being irrigated at a time
for s in slot_idx:
    m += xsum(x[f][s] for f in field_idx) <= 1

# Irrigate only when temperature is in an acceptable range

    
# wat
# offset_bef_sprinkle = [int(constraints.iloc[i]['offset_bef_sprinkle']) for i in field_id]


# In progress
# Do not irrigate between specified range
# skip_sprinkle_from: what form does it take?
# skip_sprinkle_to: what form does it take?





## Objective function (minimize)

In [8]:
# Objective function to minimize the sum of irrigation slots
m.objective = xsum(x[f][s] for f in field_idx for s in slot_idx)


## Some initial questions

What is the objective function? Are we trying to minimize the use of water? Maximize the soil content? Minimize costs? (What are costs in this case?) Maximize benefits? (what are the benefits in this case?)

# Find and send solution

In [9]:
# Use this for debugging

if min(min(humid_flag), temp_flag, min(wind_flag)) == 0:
    print('Max humidity, extreme temperature, or wind exceeds required irrigation conditions. Scheduling cancelled.')
else:
    m.max_gap = 0.05
    status = m.optimize(max_seconds=300)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(m.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        print('solution:')
        for v in m.vars:
            if abs(v.x) > 1e-6: # only printing non-zeros
                print('{} : {}'.format(v.name, v.x))


Max humidity, extreme temperature, or wind exceeds required irrigation conditions. Scheduling cancelled.


# Send solution to cloud

## For history of schedules

In [20]:
def send_schedule_to_history():
    log_dict = {}
    for f in field_idx:
        field_counter = 0
        for s in slot_idx:
            if x[f][s].x == 1:
                field_counter += 1
            if y[f][s].x == 1:
                log_dict['parcel' + str(f+1) +'_start'] = datetime.time(start_time+s//slots_hour,(s%slots_hour)*slot_dur).strftime('%H:%M')
        log_dict['parcel' + str(f+1) +'_end'] = (datetime.datetime.strptime(log_dict['parcel' + str(f+1) +'_start'], '%H:%M') + datetime.timedelta(minutes=field_counter*slot_dur)).time().strftime('%H:%M')
    log_dict['PartitionKey'] = 'Schedules'
    log_dict['RowKey'] = datetime.datetime.utcnow().date().strftime('%Y-%m-%d')
    # table_service.delete_table('Schedules')
    # table_service.create_table('Schedules')
    table_service.insert_or_replace_entity('Schedules', log_dict)

## For relay to cluster

In [21]:
def update_current_schedule():
    data_dict = []
    for f in field_idx:
        field_dict = {}
        field_dict['parcel'] = f+1
        field_counter = 0
        for s in slot_idx:
            if x[f][s].x == 1:
                field_counter += 1
            if y[f][s].x == 1:
                field_dict['start'] = datetime.time(start_time+s//slots_hour,(s%slots_hour)*slot_dur).strftime('%H:%M')
        field_dict['end'] = (datetime.datetime.strptime(field_dict['start'], '%H:%M') + datetime.timedelta(minutes=field_counter*slot_dur)).time().strftime('%H:%M')
        field_dict['schedule_date'] = datetime.datetime.utcnow().date().strftime('%Y-%m-%d')
        field_dict['PartitionKey'] = 'CurrentSchedule'
        field_dict['RowKey'] = str(field_dict['parcel'])
        data_dict.append(field_dict)
    # # table_service.delete_table('CurrentSchedule')
    # table_service.create_table('CurrentSchedule')
    for i in range(len(data_dict)):
        table_service.insert_or_replace_entity('CurrentSchedule', data_dict[i])

## For cancelled schedule

In [22]:
def send_empty_schedule():
    log_dict = {}
    data_dict = []
    for f in field_idx:
        field_dict = {}
        field_dict['parcel'] = f+1
        field_dict['start'] = '00:00'
        field_dict['end'] = '00:00'
        field_dict['PartitionKey'] = 'CurrentSchedule'
        field_dict['RowKey'] = str(field_dict['parcel'])
        field_dict['schedule_date'] = datetime.datetime.utcnow().date().strftime('%Y-%m-%d')
        data_dict.append(field_dict)
        log_dict['parcel' + str(f+1) +'_start'] = '00:00'
        log_dict['parcel' + str(f+1) +'_end'] = '00:00'
    log_dict['PartitionKey'] = 'Schedules'
    log_dict['RowKey'] = datetime.datetime.utcnow().date().strftime('%Y-%m-%d')
    table_service.insert_or_replace_entity('Schedules', log_dict)
    for i in range(len(data_dict)):
        table_service.insert_or_replace_entity('CurrentSchedule', data_dict[i])

In [23]:
# Use this for production
if min(min(humid_flag), temp_flag, min(wind_flag)) == 0:
    send_empty_schedule()
else:
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(m.objective_value))
        send_schedule_to_history()
        update_current_schedule()
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))